# Dataset Splitting

In [1]:
import os
import torch
import numpy as np
from sklearn.model_selection import train_test_split

# Set working directory and define paths for input and output data
work_dir = os.getcwd()  # Use the current directory as work_dir
input_data_dir = os.path.join(work_dir, '../Data')  # Set ../Data as input data location
output_dir = os.path.join(work_dir, '../Data')  # Set ../Data as output data location

# Define the path to the file containing labeled hypergraphs
hypergraph_file = os.path.join(input_data_dir, 'all_hypergraphs_with_labels-train.pkl')

# Load the hypergraph data from file
hypergraphs = torch.load(hypergraph_file)

# Extract labels from each hypergraph, converting each to a NumPy array for easy handling
# Each hypergraph is assumed to have a `labels` attribute for its label(s)
labels = np.array([
    hypergraph.labels.numpy() if isinstance(hypergraph.labels, torch.Tensor) else hypergraph.labels 
    for hypergraph in hypergraphs
])

# Function to perform a random train-test split
def random_train_test_split(graphs, labels, test_size=0.3, random_state=42):
    """Splits graphs and labels into training and testing sets using random split"""
    train_graphs, test_graphs, train_labels, test_labels = train_test_split(
        graphs, labels, test_size=test_size, random_state=random_state, shuffle=True)

    return train_graphs, test_graphs, train_labels, test_labels

# Split dataset into training (70%) and temporary (30%) sets
train_graphs, temp_graphs, train_labels, temp_labels = random_train_test_split(
    hypergraphs, labels, test_size=0.3, random_state=42
)

# Further split the temporary set (30%) into validation (20%) and test sets (10%)
val_graphs, test_graphs, val_labels, test_labels = random_train_test_split(
    temp_graphs, temp_labels, test_size=0.33, random_state=42
)

# Function to calculate the proportion of '1's in each label across a set of labels
def calculate_label_proportions(labels):
    proportions = np.mean(labels == 1, axis=0)  # Calculate the proportion of '1's for each label
    return proportions

# Calculate the proportion of '1's in each subset's labels
train_proportions = calculate_label_proportions(train_labels)
val_proportions = calculate_label_proportions(val_labels)
test_proportions = calculate_label_proportions(test_labels)

# Convert labels back to torch.Tensor format for PyTorch model compatibility
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

# Print the size of each subset
print(f"Training set: {len(train_graphs)} graphs")
print(f"Validation set: {len(val_graphs)} graphs")
print(f"Test set: {len(test_graphs)} graphs")

# Print the proportion of '1's for each label in each subset
print("Proportion of '1's for each label in training set:", train_proportions)
print("Proportion of '1's for each label in validation set:", val_proportions)
print("Proportion of '1's for each label in test set:", test_proportions)


Training set: 4256 graphs
Validation set: 1222 graphs
Test set: 602 graphs
Proportion of '1's for each label in training set: [0.06343985 0.06414474 0.57307331 0.10103383 0.30451128]
Proportion of '1's for each label in validation set: [0.06873977 0.06792144 0.57774141 0.09165303 0.30032733]
Proportion of '1's for each label in test set: [0.04318937 0.06644518 0.57807309 0.09966777 0.3255814 ]


# Hypergraph Neural Network Model Architecture

In [ ]:
import torch
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, recall_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set random seed for reproducibility across random, numpy, and PyTorch functions
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

set_seed(42)  # Set seed to 42 for consistency

# Define a hypergraph convolution layer with attention mechanism
class HypergraphAttentionConv(nn.Module):
    def __init__(self, in_features, out_features, num_heads=1):
        super(HypergraphAttentionConv, self).__init__()
        self.num_heads = num_heads  # Number of attention heads
        # Create multiple attention heads, each mapping input to output features
        self.attention_heads = nn.ModuleList([nn.Linear(in_features, out_features) for _ in range(num_heads)])
        self.fc = nn.Linear(out_features, out_features)  # Final linear layer to combine head outputs

    def forward(self, x, H):
        """
        Forward pass through the attention-based hypergraph convolution.
        - x: Node feature matrix
        - H: Hypergraph incidence matrix
        """
        hyperedge_features = []
        attn_weights = []
        
        # Process each attention head
        for attn_head in self.attention_heads:
            # Compute hyperedge features with attention
            hyperedge_feat = torch.matmul(H.T, F.elu(attn_head(x)))  # Apply attention head, then non-linearity
            hyperedge_features.append(hyperedge_feat)

            # Calculate attention weights
            attn_weight = F.softmax(torch.matmul(H.T, attn_head(x)), dim=0)
            attn_weights.append(attn_weight)

        # Aggregate hyperedge features across heads by averaging
        hyperedge_features = torch.stack(hyperedge_features, dim=0).mean(dim=0)
        
        # Compute node features by multiplying with incidence matrix and final linear layer
        x = torch.matmul(H, hyperedge_features)
        x = self.fc(x)  # Final output layer
        return x, torch.mean(torch.stack(attn_weights), dim=0)  # Return output and average attention weights

# Define the HGNN model with two hypergraph convolutional layers and attention
class TwoLayerHGNNWithAttention(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, dropout=0.5, num_heads=8, dosage_weight=1.0):
        super(TwoLayerHGNNWithAttention, self).__init__()
        self.dosage_weight = dosage_weight  # Weight for dosage feature
        # First hypergraph attention layer
        self.conv1 = HypergraphAttentionConv(in_features, hidden_features, num_heads=num_heads)
        self.dropout1 = nn.Dropout(dropout)  # Dropout for regularization
        # Second hypergraph attention layer
        self.conv2 = HypergraphAttentionConv(hidden_features, hidden_features, num_heads=1)
        self.dropout2 = nn.Dropout(dropout)  # Dropout for regularization
        self.fc = nn.Linear(hidden_features, out_features)  # Fully connected output layer

    def forward(self, x, H):
        """
        Forward pass for the HGNN model.
        - x: Node feature matrix
        - H: Hypergraph incidence matrix
        """
        # Apply dosage weight to feature at index 90
        x[:, 90] = torch.clamp(x[:, 90] * self.dosage_weight, min=0, max=10)
        
        # First attention layer with activation and dropout
        x, attn_weights1 = self.conv1(x, H)
        x = F.relu(x)
        x = self.dropout1(x)

        # Second attention layer with activation and dropout
        x, attn_weights2 = self.conv2(x, H)
        x = F.relu(x)
        x = self.dropout2(x)

        # Apply global average pooling to reduce features to a single vector
        x = torch.mean(x, dim=0)  # Output shape: [hidden_features]

        # Final fully connected layer for output prediction
        x = self.fc(x)  # Output shape: [out_features]
        return x, attn_weights1, attn_weights2  # Return output and attention weights from both layers

# Model parameters
in_features = 91          # Input feature dimension
hidden_features = 96      # Hidden layer dimension
out_features = 5          # Output dimension (e.g., number of classes)
dropout = 0.5             # Dropout rate
num_heads = 4             # Number of attention heads
dosage_weight = 1.0       # Weight factor for dosage feature

# Instantiate the HGNN model
model = TwoLayerHGNNWithAttention(in_features, hidden_features, out_features, dropout, num_heads, dosage_weight)
print(model)


# Model Training

In [ ]:
import torch
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, recall_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set class weights to handle label imbalance
def compute_class_weights(labels):
    num_classes = labels.size(1)
    pos_counts = labels.sum(dim=0)
    neg_counts = labels.size(0) - pos_counts
    pos_weight = neg_counts / (pos_counts + 1e-6)  # Avoid division by zero with small constant
    return pos_weight

# Define loss function, optimizer, and learning rate scheduler
pos_weight = compute_class_weights(train_labels)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)  # Use BCEWithLogitsLoss for multi-label classification
learning_rate = 0.0005
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)  # Reduce LR on plateau in validation loss

# Early stopping mechanism to halt training if validation loss does not improve
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss  # Update best loss
            self.counter = 0  # Reset counter if validation loss improves
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True  # Trigger early stopping if patience is exceeded

early_stopping = EarlyStopping(patience=5, min_delta=0.001)

# Construct hypergraph incidence matrix for a given hypergraph
def construct_hypergraph_incidence_matrix(hypergraph):
    node_indices, edge_indices, values = [], [], []
    node_dict = {node: idx for idx, node in enumerate(hypergraph.nodes)}  # Map nodes to indices
    for j, edge in enumerate(hypergraph.edges):
        for node in hypergraph.edges[edge]:
            i = node_dict[node]
            node_indices.append(i)
            edge_indices.append(j)
            values.append(1.0)  # Set connection value to 1.0
    indices = torch.LongTensor([node_indices, edge_indices])
    values = torch.FloatTensor(values)
    H = torch.sparse_coo_tensor(indices, values, (len(hypergraph.nodes), len(hypergraph.edges)))
    return H

# Generator function to yield batches of data and labels
def batchify(data, labels, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size], labels[i:i + batch_size]

# Print the current learning rate from the optimizer
def print_learning_rate(optimizer):
    for param_group in optimizer.param_groups:
        print(f"Current Learning Rate: {param_group['lr']}")

# Train model for a single epoch
def train_epoch(graphs, labels, batch_size, loss_fn, grad_clip_value=1.0):
    model.train()  # Set model to training mode
    total_loss = 0
    batches = list(batchify(graphs, labels, batch_size))
    pbar = tqdm(total=len(batches), desc="Training")
    
    for batch_graphs, batch_labels in batches:
        optimizer.zero_grad()  # Clear gradients
        batch_loss = 0
        for i, graph in enumerate(batch_graphs):
            # Stack node features and construct incidence matrix
            x = torch.stack([graph.node_features[node].float() for node in graph.nodes])
            H = construct_hypergraph_incidence_matrix(graph)
            output, attn_weights1, attn_weights2 = model(x, H)
            
            # Compute loss between model output and labels
            loss = loss_fn(output.view(1, -1), batch_labels[i].view(1, -1).float())
            batch_loss += loss
        batch_loss.backward()  # Backpropagate loss
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_value)  # Clip gradients
        optimizer.step()  # Update parameters
        total_loss += batch_loss.item()
        pbar.update(1)
    
    pbar.close()
    return total_loss / len(graphs)  # Return average loss for the epoch

# Validation function to evaluate model on validation set
def validate(graphs, labels, loss_fn):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    all_outputs = []
    all_labels = []
    
    with torch.no_grad():  # Disable gradient computation
        for i, graph in enumerate(graphs):
            # Stack node features and construct incidence matrix
            x = torch.stack([graph.node_features[node].float() for node in graph.nodes])
            H = construct_hypergraph_incidence_matrix(graph)
            output, attn_weights1, attn_weights2 = model(x, H)
            
            # Compute validation loss
            loss = loss_fn(output.view(1, -1), labels[i].view(1, -1).float())
            total_loss += loss.item()
            all_outputs.append(torch.sigmoid(output).cpu().numpy())
            all_labels.append(labels[i].cpu().numpy())

    all_outputs = np.vstack(all_outputs)  # Stack all outputs
    all_labels = np.vstack(all_labels)  # Stack all labels
    
    # Calculate recall and F1 score
    recall = recall_score(all_labels, all_outputs.round(), average='micro')
    f1 = f1_score(all_labels, all_outputs.round(), average='micro')
    
    return total_loss / len(graphs), recall, f1

# Wrapper function to train and validate the model across epochs
def train_model(train_graphs, train_labels, val_graphs, val_labels, model, loss_fn, optimizer, scheduler, num_epochs=50, batch_size=16):
    train_losses, val_losses, val_recalls, val_f1s = [], [], [], []
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train_loss = train_epoch(train_graphs, train_labels, batch_size, loss_fn)  # Train for one epoch
        train_losses.append(train_loss)
        
        # Validate the model
        val_loss, val_recall, val_f1 = validate(val_graphs, val_labels, loss_fn)
        val_losses.append(val_loss)
        val_recalls.append(val_recall)
        val_f1s.append(val_f1)
        
        scheduler.step(val_loss)  # Adjust learning rate based on validation loss
        print(f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
              f"Validation Recall: {val_recall:.4f}, Validation F1: {val_f1:.4f}")

        print_learning_rate(optimizer)  # Print current learning rate

        early_stopping(val_loss)  # Check for early stopping
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

# Train the model
train_model(train_graphs, train_labels, val_graphs, val_labels, model, loss_fn, optimizer, scheduler, num_epochs=30, batch_size=64)


# Model Evaluation

In [ ]:
import torch
import numpy as np
from sklearn.metrics import roc_curve, auc, precision_recall_fscore_support, accuracy_score, confusion_matrix
import pandas as pd
import os
from matplotlib import rcParams

# Set global font to Arial for consistent plotting
rcParams['font.family'] = 'Arial'

# Function to evaluate the model on a specific dataset
def evaluate_model(graphs, labels, model, output_dir, data_name):
    # Set the model to evaluation mode
    model.eval()

    # Collect predictions and labels for the specified dataset
    all_outputs = []
    all_labels = []

    # Disable gradient calculations for evaluation
    with torch.no_grad():
        for i, graph in enumerate(graphs):
            x = torch.stack([graph.node_features[node] for node in graph.nodes])  # Extract node features
            H = construct_hypergraph_incidence_matrix(graph)  # Construct hypergraph incidence matrix
            output, attn_weights1, _ = model(x, H)  # Get model output, taking only `output`
            
            all_outputs.append(output.cpu().numpy())
            all_labels.append(labels[i].cpu().numpy())

    # Convert predictions and labels to NumPy arrays
    final_outputs = np.vstack(all_outputs)
    final_labels = np.vstack(all_labels)

    # Compute and save evaluation metrics
    compute_and_save_metrics(final_labels, final_outputs, output_dir, data_name)


# Function to compute and save various performance metrics and ROC data
def compute_and_save_metrics(labels, outputs, output_dir, data_name):
    num_classes = labels.shape[1]
    metrics = {
        'Class': [],
        'Precision': [],
        'Recall': [],
        'F1 Score': [],
        'AUC': [],
        'Accuracy': [],
        'Specificity': []
    }
    roc_data_long_format = {'Class': [], 'Reference': [], 'Predicted': []}
    
    for i in range(num_classes):
        # Apply sigmoid to convert logits to probabilities
        probabilities = torch.sigmoid(torch.tensor(outputs))  # Convert logits to probabilities in range 0-1
        
        # ROC curve and AUC calculation
        fpr, tpr, thresholds = roc_curve(labels[:, i], probabilities[:, i].numpy())
        roc_auc = auc(fpr, tpr)
        
        # Save true labels and predicted probabilities for ROC data in long format
        for ref, pred in zip(labels[:, i], probabilities[:, i].numpy()):
            roc_data_long_format['Class'].append(f'Class_{i+1}')
            roc_data_long_format['Reference'].append(ref)
            roc_data_long_format['Predicted'].append(pred)
        
        # Calculate Precision, Recall, F1 Score, Accuracy, and Specificity
        pred_binary = (probabilities[:, i] > 0.5).numpy().astype(int)  # Use threshold 0.5 for binary predictions
        precision, recall, f1, _ = precision_recall_fscore_support(labels[:, i], pred_binary, average='binary')
        accuracy = accuracy_score(labels[:, i], pred_binary)
        
        tn, fp, fn, tp = confusion_matrix(labels[:, i], pred_binary).ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # Prevent division by zero

        # Save metrics for the current class
        metrics['Class'].append(f'Class_{i+1}')
        metrics['Precision'].append(precision)
        metrics['Recall'].append(recall)
        metrics['F1 Score'].append(f1)
        metrics['AUC'].append(roc_auc)
        metrics['Accuracy'].append(accuracy)
        metrics['Specificity'].append(specificity)
    
    # Compute average metrics across all classes
    avg_metrics = {
        'Class': ['Average'],
        'Precision': [np.mean(metrics['Precision'])],
        'Recall': [np.mean(metrics['Recall'])],
        'F1 Score': [np.mean(metrics['F1 Score'])],
        'AUC': [np.mean(metrics['AUC'])],
        'Accuracy': [np.mean(metrics['Accuracy'])],
        'Specificity': [np.mean(metrics['Specificity'])]
    }
    
    # Add average metrics to the metrics dictionary
    for key in metrics:
        metrics[key].append(avg_metrics[key][0])
    
    # Save ROC data in long format to CSV
    roc_df_long = pd.DataFrame(roc_data_long_format)
    roc_df_long.to_csv(os.path.join(output_dir, f'{data_name}_roc_data_hypergraph.csv'), index=False)

    # Save evaluation metrics to CSV
    metrics_df = pd.DataFrame(metrics)
    metrics_df.to_csv(os.path.join(output_dir, f'{data_name}_metrics_hypergraph.csv'), index=False)
    print(f"Metrics and ROC data saved to {output_dir}.")


# Set working directory and define input/output paths
work_dir = os.getcwd()  # Use the current directory as work_dir
input_data_dir = os.path.join(work_dir, '../Data')  # Set ../Data as input data location
output_dir = os.path.join(work_dir, '../Data')  # Set ../Data as output data location

# Evaluate model on training set
#evaluate_model(train_graphs, train_labels, model, output_dir, "train")

# Evaluate model on validation set
#evaluate_model(val_graphs, val_labels, model, output_dir, "validation")

# Evaluate model on test set
evaluate_model(test_graphs, test_labels, model, output_dir, "test")
